## Old

### Importieren der Klassen

In [ ]:
import time
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import bitsandbytes as bnb
import accelerate

### Festlegen des Modells

In [ ]:
model_id="/home/thsch026/masterarbeit/models/generated/llm-awq/Meta-Llama-3-8B-Instruct-HF-AWQ-4bit"

### Initialisieren des Modells

In [ ]:

tokenizer = AutoTokenizer.from_pretrained(model_id)
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=quantization_config, #load_in_8bit=True,#
    device_map='auto',
    token='<EOL>'
)
#peft_model = "BojanaBas/Meta-Llama-3-8B-Instruct-pqa-10"
#model = PeftModel.from_pretrained(base_model, peft_model)

### Inferenz Test

### Autoawq library is needed to test inference with AWQ Models

In [ ]:
pip install autoawq

In [ ]:
# Eingabetext
input_text = "Once upon a time"
#input_text = "Why should I eat healthy?"

# Tokenisiere den Eingabetext
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Messe die Zeit vor der Inferenz
start_time = time.time()

# Führe die Inferenz durch
outputs = model.generate(input_ids, max_length=50)

# Messe die Zeit nach der Inferenz
end_time = time.time()

# Berechne die Inferenz-Zeit
inference_time = end_time - start_time

# Dekodiere die Ausgabe
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
number_of_words = len(output_text.split())
number_of_tokens = len(outputs[0])

token_per_second = number_of_tokens/inference_time

print(f"Inferenz-Zeit: {inference_time} Sekunden")
print(f"Output: {output_text}")
print("Tokens per second: ", token_per_second)



In [ ]:
import csv

# Die neuen Daten, die hinzugefügt werden sollen
neue_daten = ['Lukas', 28]

# Öffne die Datei im Anhangsmodus ('a' steht für append)
with open('daten.csv', mode='a', newline='') as file:
    writer = csv.writer(file)
    
    # Schreibe die neuen Daten in die CSV-Datei
    writer.writerow(neue_daten)

print("Neue Daten erfolgreich hinzugefügt.")

In [ ]:
pip install sentencepiece

## Evaluierung Modelle

### Sentencepiece muss ggf. noch mal installiert werden für Mistral (fast tokenizer)

In [ ]:
pip install sentencepiece

### Laden der libaries und festlegen der GPU

In [1]:
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:256"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
!echo $CUDA_VISIBLE_DEVICES
# torch.cuda.set_device(1)

1


In [2]:
import time
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import bitsandbytes as bnb
import accelerate
import csv

### Ergebnis Dateien

In [3]:
full_csv= "/home/thsch026/masterarbeit/Slim/results/inference/inference_full.csv"
data_csv= "/home/thsch026/masterarbeit/Slim/results/inference/inference_data_only.csv"

##  1. Referenz Modelle

### Llama-3-8B-Instruct

In [ ]:
model_id ="Meta-Llama-3-8B-Instruct"
model_loc = "meta-llama/Meta-Llama-3-8B-Instruct"

### Mistral-7B-Instruct-v0.2

In [ ]:
model_id = "Mistral-7B-Instruct-v0.2"
model_loc = "mistralai/Mistral-7B-Instruct-v0.2"

### Llama-2-7b-chat-hf

In [ ]:
model_id = "Llama-2-7b-chat-hf"
model_loc = "meta-llama/Llama-2-7b-chat-hf"

## 2. AWQ Modelle

### Llama-3-8B-Instruct-HF-AWQ-4bit

In [ ]:
model_path = "/home/thsch026/masterarbeit/models/generated/llm-awq/"
model_id = "Meta-Llama-3-8B-Instruct-HF-AWQ-4bit"
model_loc = model_path + model_id

### Mistral-7B-Instruct-v0.2-AWQ-4bit

In [ ]:
model_path = "/home/thsch026/masterarbeit/models/generated/llm-awq/"
model_id = "Mistral-7B-Instruct-v0.2-AWQ-4bit"
model_loc = model_path + model_id

### Llama-2-7b-chat-HF-AWQ-4bit

In [ ]:
model_path = "/home/thsch026/masterarbeit/models/generated/llm-awq/"
model_id = "Llama-2-7b-chat-HF-AWQ-4bit"
model_loc = model_path + model_id

In [ ]:
model_path = "/home/thsch026/masterarbeit/models/generated/awq/quant/"
model_id = "Meta-Llama-3-8B-Instruct-HF-w8-g128-awq-v2.pt"
model_loc = model_path + model_id

## 3. Pruned Models (shortened-LLM)

### Meta-Llama-3-8B

In [ ]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune/shortened-llm/Meta-Llama-3-8B"
model_id = "Meta-Llama-3-8B_pruned_llm-shortened"

### Llama-2-7b-hf (schlechtes Ergebnis)

In [ ]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune/shortened-llm/Llama-2-7b-hf"
model_id = "Llama-2-7b-hf_pruned_llm-shortened"

### Mistral-7B-Instruct

In [ ]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune/shortened-llm/Mistral-7B-Instruct"
model_id = "Mistral-7B-Instruct_pruned_llm-shortened"

## 4. Lora Models

#### Meta-Llama-3-8B_ms-marco (Kein gutes Ergebnis)

In [12]:
model_loc = "/home/thsch026/masterarbeit/models/generated/lora/Meta-Llama-3-8B_ms-marco/model"
model_id = "Meta-Llama-3-8B_ms-marco"

##  5. Qlora Models

### Meta-Llama-3-8B-Instruct_qlora

In [ ]:
model_loc = "/home/thsch026/masterarbeit/models/generated/qlora/Meta-Llama-3-8B-Instruct_qlora"
model_id = "Meta-Llama-3-8B-Instruct_qlora"

### Mistral-7B-Instruct-v0.2_qlora

In [ ]:
model_loc = "/home/thsch026/masterarbeit/models/generated/qlora/Mistral-7B-Instruct-v0.2_qlora"
model_id = "Mistral-7B-Instruct-v0.2_qlora"

### llama-2-7b-chat_qlora (Fehlt noch)

In [ ]:
model_loc = "/home/thsch026/masterarbeit/models/generated/qlora/llama-2-7b-chat_qlora"
model_id = "llama-2-7b-chat_qlora"

## 6. Prune + Lora Models (shortened-LLM)

#### Meta-Llama-3-8B

In [4]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora/shortened-llm/Meta-Llama-3-8B_prune_lora"
model_id = "Meta-Llama-3-8B_prune_lora"

#### Llama-2-7b-hf_prune_lora

In [8]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora/shortened-llm/Llama-2-7b-hf_prune_lora"
model_id = "Llama-2-7b-hf_prune_lora"

#### Mistral-7B-Instruct_prune_lora

In [4]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora/shortened-llm/Mistral-7B-Instruct_prune_lora"
model_id = "Mistral-7B-Instruct_prune_lora"

## 7. Prune + Lora + AWQ Models

#### Mistral-7B-Instruct_prune_lora_awq

In [4]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora_plus_awq/Mistral-7B-Instruct-v02_prune_lora_awq"
model_id = "Mistral-7B-Instruct_prune_lora_awq"

#### Meta-Llama-3-8B_prune_lora_awq

In [8]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora_plus_awq/Meta-Llama-3-8B_prune_lora_awq"
model_id ="Meta-Llama-3-8B_prune_lora_awq"

#### Llama-2-7b-chat-hf_prune_lora_awq

In [12]:
model_loc = "/home/thsch026/masterarbeit/models/generated/prune_plus_lora_plus_awq/Llama-2-7b-chat-hf_prune_lora_awq"
model_id ="Llama-2-7b-chat-hf_prune_lora_awq"

## Inference Task

### Laden des Modells

In [13]:

tokenizer = AutoTokenizer.from_pretrained(model_loc)

#quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)
#quantization_config = BitsAndBytesConfig(bnb_4bit_compute_dtype=torch.bfloat16)

model = AutoModelForCausalLM.from_pretrained(
    model_loc,
    trust_remote_code=True,
    # Bei AWQ Modellen keine quantization config angeben
   # quantization_config=quantization_config, #load_in_8bit=True
    device_map='cuda'
)


You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


### Interferenz mit Ergebnis Speicherung

In [14]:
# Funktion für das Speichern der Werte

def append_data(result):
    
    with open(full_csv, mode='a', newline='') as file:
        full_writer = csv.writer(file)
        full_writer.writerow(result)
        
    with open(data_csv, mode='a', newline='') as file:
        data_writer = csv.writer(file)
        data_writer.writerow(result[0:4])

                
device = 'cuda'

question_list = ["Once upon a time", "Why should I eat healthy?", "Is there a second life after death?", "What is the most common injury?","Who is Konrad Zuse?"]

for question in question_list:
    input_ids = tokenizer.encode(question, return_tensors='pt')
    # Änderung dr Zeile weil es Probleme mit der Ausführung gab (cuda/cpu anomalie)
    input_ids = tokenizer(question, return_tensors="pt").input_ids.to(device)
    input_ids.to('cuda:0')
    start_time = time.time()

    # Führe die Inferenz durch
    outputs = model.generate(input_ids, max_length=200)

    # Messe die Zeit nach der Inferenz
    end_time = time.time()

    # Berechne die Inferenz-Zeit
    inference_time = end_time - start_time

    # Dekodiere die Ausgabe
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Berchnung des Ergebnis
    #number_of_words = len(output_text.split())
    number_of_tokens = len(outputs[0])
    memory_usage = round((torch.cuda.memory_allocated() / (1024 **3)),2)
    token_per_second = number_of_tokens/inference_time
    result = [model_id,token_per_second, inference_time, memory_usage, question, output_text]
    print(result)
    append_data(result)
              
    
                                                                
    

    
    

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['Llama-2-7b-chat-hf_prune_lora_awq', 3.1914819509381283, 62.66681218147278, 3.13, 'Once upon a time', 'Once upon a time, there was a poor farmer named J. Hinweis desktops were used for many years. However, with the advent of portable computing and mobile devices, the usage shifted towards laptops, handheld devices and tablets. Now, the trend is moving towards virtual and augmented reality devices and IoT (Internet of Things) devices for remote working and home automation.\n\nLetting a software monopolist develop something of great importance like a voting device means giving them the power to manipulate it as they see fit. The internet has made inter-regulatory compliance less about "foreign control" and more about security at the perimeter and around the edge. But not every device is trustworthy; some are hacked by malware attacks which can influence the outcome of the votes.\n\nTo ensure the integrity and trustworthiness of voting systems, these must be designed and designed-conduct

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['Llama-2-7b-chat-hf_prune_lora_awq', 3.3487477940305626, 59.723816871643066, 3.13, 'Why should I eat healthy?', 'Why should I eat healthy?\n Unterscheidung zwischen Eating healthy and eating cravings\nWhy should I eat healthy?\n"’ll die’ll die’ll never ever try: the end of the story, one day’.” This is not the ending you’re looking for, I imagine! Instead, here are some reasons for living, especially when it comes to eating:\nEating healthy is the best way to maintain good physical and mental health. One great way to do so is by eating a balance of nutrient-rich foods such as fruits, vegetables, whole grains, and lean proteins.\nThe Foods to Eat\nFruits and Vegetables: Both are great sources of essential vitamins, minerals and fiber.\nFruits: Provides nutrient-rich nuggets full of vitamins, minervs, antioxidants,']


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['Llama-2-7b-chat-hf_prune_lora_awq', 3.4837896382184175, 33.29707360267639, 3.13, 'Is there a second life after death?', 'Is there a second life after death?\n nobody knows\n\nIn religion and spirituality, there are beliefs and theories about the possibility of a second life or afterlife. The concept of eternal life, immortality or even reincarnation vary among different religions, depending on their teachings. Overall, the existence of a second life remains a topic of speculation and argument for those who are skeptical about the nature of existence, and others who believe their soul or consciousness will carry out existence after a physical life is lived.']


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['Llama-2-7b-chat-hf_prune_lora_awq', 3.4295234987689414, 36.739797830581665, 3.13, 'What is the most common injury?', 'What is the most common injury?\n Hinweis - This article discusses the most common injuries, particularly in the workplace or on and off the job site. Some injuries, such as fractures and head injuries, can happen suddenly due to falls, accidents, and unavoidable physical or natural causes, as mentioned herein. However, these are very rare and likely to occur due to unavoidable situations. However, as mentioned above, the most common injuries are from repetitive stress and sprains, and all care should be taken to avoid any sudden injuries.']
['Llama-2-7b-chat-hf_prune_lora_awq', 3.347256333181027, 59.750428438186646, 3.13, 'Who is Konrad Zuse?', 'Who is Konrad Zuse?\n✿ Ingridhöistock.com 🎓\nKonrad Zuse is an German inventor and computer enthusiast who is widely recognized for being the first person to write a program that built a calculating machine and implement it.\

In [11]:
del model # Specify variable
torch.cuda.empty_cache()